# Account-Tax Pipeline Data Exploration Template

주피터 노트북에서 `account-tax` 파이프라인의 데이터를 로드하고 탐색하는 템플릿입니다.

## 1. 기본 라이브러리 임포트

In [ ]:
# 데이터 처리
import pandas as pd
import numpy as np

# 데이터 로드
import pickle
import json
from pathlib import Path

# Kedro 통합
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# HuggingFace datasets
from datasets import Dataset, DatasetDict

# 화면 표시 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## 2. Kedro Catalog을 통한 데이터 로드 (권장)

In [ ]:
# 프로젝트 부트스트랩
project_path = Path.cwd().parent  # notebooks/에서 실행 시
bootstrap_project(project_path)

# Kedro 세션 생성
with KedroSession.create(project_path=project_path) as session:
    context = session.load_context()
    catalog = context.catalog

    # 데이터셋 로드
    raw_data = catalog.load("raw_account_data")  # Parquet
    standardized_data = catalog.load("standardized_data")  # Parquet
    validated_data = catalog.load("validated_data")  # Parquet
    base_table = catalog.load("base_table")  # Parquet (features 포함)

## 3. 기본 데이터 탐색

In [ ]:
# 데이터 형태 확인
print(f"Shape: {base_table.shape}")
print(f"Columns: {base_table.columns.tolist()}")
print(f"Memory: {base_table.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 기본 통계
base_table.info()

In [ ]:
# 처음 몇 행 확인
base_table.head()

In [ ]:
# 데이터 타입 확인
base_table.dtypes

In [ ]:
# 결측치 확인
base_table.isnull().sum()

## 4. HuggingFace Dataset 로드 및 탐색

In [ ]:
# Pickle 파일 로드
with open('../data/05_model_input/serialized_datasets.pkl', 'rb') as f:
    datasets = pickle.load(f)

# 구조 확인
print(f"Type: {type(datasets)}")
print(datasets)

In [ ]:
# 각 split 정보 확인
for split_name, dataset in datasets.items():
    print(f"\n{split_name}:")
    print(f"  Rows: {len(dataset)}")
    print(f"  Columns: {dataset.column_names}")
    print(f"  Features: {dataset.features}")

In [ ]:
# 샘플 데이터 확인
print("Train set sample:")
print(datasets['train'][0])

## 5. 라벨 분포 분석

In [ ]:
# DataFrame으로 변환 (전체 또는 샘플)
train_df = datasets['train'].to_pandas()
# train_df = datasets['train'].select(range(10000)).to_pandas()  # 샘플링

# 라벨 분포
label_counts = train_df['labels'].value_counts()
print(f"\nLabel distribution (top 10):")
print(label_counts.head(10))

In [ ]:
# 라벨 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 상위 20개 라벨
label_counts.head(20).plot(kind='bar', ax=axes[0])
axes[0].set_title('Top 20 Label Distribution')
axes[0].set_xlabel('Label')
axes[0].set_ylabel('Count')

# 텍스트 길이 분포
text_lengths = train_df['text'].str.len()
axes[1].hist(text_lengths, bins=50, edgecolor='black')
axes[1].set_title('Text Length Distribution')
axes[1].set_xlabel('Character Count')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 6. 무작위 샘플 확인

In [ ]:
# 무작위 샘플 추출
import random
random.seed(42)
indices = random.sample(range(len(datasets['train'])), k=5)
samples = datasets['train'].select(indices)

# 샘플 표시
for i, sample in enumerate(samples):
    print(f"\n{'='*80}")
    print(f"Sample {i+1}")
    print(f"{'='*80}")
    print(f"Text: {sample['text']}")
    print(f"Label: {sample['labels']}")

## 7. Token Length Report 확인

In [ ]:
# JSON 리포트 로드
with open('../data/08_reporting/token_length_report.json', 'r') as f:
    report = json.load(f)

# 예쁘게 출력
import pprint
pprint.pprint(report)

In [ ]:
# 주요 통계 확인
print(f"Overall mean token length: {report['overall']['mean']}")
print(f"Overall max token length: {report['overall']['max']}")
print(f"\nTrain split stats:")
print(f"  Mean: {report['per_split']['train']['mean']}")
print(f"  Max: {report['per_split']['train']['max']}")

## 8. 커스텀 분석 영역

아래에 추가 분석 코드를 작성하세요.

In [ ]:
# 여기에 분석 코드 작성
